# 1 网格搜索寻优

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.base import BaseEstimator

def find_best_params_GridSearchCV(model, param_grid, X, y, cv=5, scoring='accuracy'):
    """
    使用GridSearchCV寻找模型的最佳参数

    参数:
    model: 待优化的模型实例,必须是sklearn.BaseEstimator的子类
    param_grid: 参数网格，字典格式，键是参数名，值是参数取值列表,参数格式如下
        # param_grid = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'], 'kernel': ['rbf', 'linear']}
    X: 训练数据特征,二维数组或DataFrame
    y: 训练数据标签,一维数组或Series
    cv: 可选,默认为5,交叉验证的折数
    scoring: 可选，默认为'accuracy'，评估指标

    返回:
    grid: GridSearchCV对象,包含最佳参数和最佳得分等信息

    抛出:
    TypeError: 如果输入参数的类型不正确
    Exception: 如果在模型训练过程中发生错误
    """

# 1. `estimator`：要优化的 estimator 对象，即模型对象。
# 2. `param_grid`：一个字典，用于指定要搜索的超参数及其可能的值。
#     例如，`param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1]}` 表示要搜索的两个超参数 `C` 和 `gamma`，
#     它们的值分别为 0.1、1 和 10。
# 3. `scoring`：一个字符串或一个 scoring 对象，用于指定评估模型性能的指标。例如，`scoring = 'accuracy'` 
#     表示使用准确率作为评估指标。
# 4. `cv`：一个整数或一个 cross-validation 策略对象，用于指定交叉验证的策略。例如，`cv = 5` 表示使用 5 折交叉验证。
# 5. `n_jobs`：一个整数，用于指定并行计算的工作进程数。例如，`n_jobs = -1` 表示使用所有的 CPU 核心进行并行计算。
# 6. `verbose`：一个整数，用于指定日志的详细程度。例如，`verbose = 2` 表示输出更多的日志信息。
# 7. `pre_dispatch`：一个字符串或一个整数，用于指定预分配给每个工作进程的计算资源。
#     例如，`pre_dispatch = '2*n_jobs'` 表示每个工作进程将获得 2 倍的计算资源。
# 8. `random_state`：一个整数，用于指定随机数种子。

    # 检查输入数据类型和大小
    X, y = check_X_y(X, y)
    
    # 检查模型是否为BaseEstimator的子类,检查param_grid是否为字典
    if not isinstance(model, BaseEstimator):
        raise TypeError('model must be an instance of sklearn.BaseEstimator')

    if not isinstance(param_grid, dict):
        raise TypeError('param_grid must be a dictionary')

    # 创建并训练GridSearchCV对象
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring=scoring, n_jobs=-1)
    
    try:
        grid_search.fit(X, y)
    except Exception as e:
        raise Exception(f"An error occurred during fitting: {e}")

    # 确保模型已成功训练
    check_is_fitted(grid_search, 'best_estimator_')

    # print(f"Best parameters: {grid_search.best_params_}")
    # print(f"Best score: {grid_search.best_score_}")
    
    return grid_search

# 2 随机搜索寻优

随机寻优可以使用网格参数，但是这样会失去随机寻优的优势。随机寻优的核心在于它不是穷举所有可能的参数组合，而是从参数空间中随机选择组合进行尝试。
这种方法在参数空间较大时更加高效，因为它不会测试所有组合，而是通过随机采样来探索参数空间。
如果你在随机寻优中使用网格参数，即指定参数的固定值而不是分布，那么你实际上是在进行一种特殊的随机寻优，它可能会更接近于网格寻优的行为。
这样做不会充分利用随机寻优的灵活性，因为你没有指定参数的分布，而是给出了一个固定的值列表。
例如，如果你在随机寻优中设置了参数如下：
```python
param_dist = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    # ... 其他参数 ...
}
```
在这种情况下，`n_estimators` 和 `max_depth` 的可能取值是固定的，而不是随机的。每次迭代时，`RandomizedSearchCV` 将从这些固定的值中随机选择，而不是从可能的分布中采样。这可能会导致搜索过程更接近于网格寻优，因为你实际上是在一个有限的网格上随机选择点。
如果你想充分利用随机寻优的优势，你应该为每个参数定义一个分布，例如：
```python
param_dist = {
    'n_estimators': randint(10, 200),  # 从10到200的整数
    'max_depth': randint(1, 30),       # 从1到30的整数，包括None
    # ... 其他参数 ...
}
```
在这个例子中，`randint` 分布将允许 `n_estimators` 从10到200的整数中随机选择，而 `max_depth` 可以从1到30的整数中随机选择，包括 `None`。这样，每次迭代都会从这些分布中随机采样参数值，从而更有效地探索参数空间。


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.validation import check_X_y, check_is_fitted

def find_best_params_RandomizedSearchCV(model, param_grid, X, y, cv=5, scoring='accuracy', n_iter=10):
    """
    使用RandomizedSearchCV寻找模型的最佳参数

    参数:
    model: 待优化的模型实例,必须是sklearn.BaseEstimator的子类
    param_grid: 参数空间，字典格式，键是参数名，值是参数取值分布,参数格式如下
        # from scipy.stats import randint, uniform
        # param_dist = {
        #'n_estimators': randint(50, 200),  # 从50到200的整数
        #'max_depth': randint(2, 10),       # 从2到10的整数
        #'min_samples_split': randint(2, 11),  # 从2到10的整数
        #'min_samples_leaf': randint(1, 11),   # 从1到10的整数
        #'max_features': uniform(0, 1),        # 从0到1的均匀分布
        # }

    X: 训练数据特征,二维数组或DataFrame
    y: 训练数据标签,一维数组或Series
    cv: 可选,默认为5,交叉验证的折数
    scoring: 可选，默认为'accuracy'，评估指标
    n_iter: 可选,默认为10,随机搜索的迭代次数

    返回:
    random_search: RandomizedSearchCV对象,包含最佳参数和最佳得分等信息

    抛出:
    TypeError: 如果输入参数的类型不正确
    Exception: 如果在模型训练过程中发生错误
    """
    
    # 检查输入数据类型和大小
    X, y = check_X_y(X, y)
    # 检查模型是否为BaseEstimator的子类,检查param_grid是否为字典
    if not isinstance(model, BaseEstimator):
        raise TypeError('model must be an instance of sklearn.BaseEstimator')
        
    # 检查param_grid是否为字典
    if not isinstance(param_grid, dict):
        raise TypeError('param_grid must be a dictionary')

    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=n_iter,
        cv=cv,
        scoring=scoring
    )
    
    try:
        random_search.fit(X, y)
    except Exception as e:
        raise Exception(f"An error occurred during fitting: {e}")

    # 确保模型已成功训练
    check_is_fitted(random_search, 'best_estimator_')
    return random_search